# Homework 3: Interactive visualisation

In [24]:
import os
import pandas as pd
import json
import folium
import numpy as np
from IPython.core.display import display,HTML

# package to define colormaps
import branca.colormap as cm

## Question 1: Unemployment rate in Europe

We import the topology of European countries. We delete Israel from the country list because according to https://en.wikipedia.org/wiki/List_of_European_countries_by_population, it is not in Europe.

We use the data of http://appsso.eurostat.ec.europa.eu/nui/show.do?dataset=une_rt_a&lang=en about the percentage of unemployment in the active population of each country. We clean the data so all country names are well matched between the database and the topology.

We think the percentage in the active population is more representative of what unemployment is, because if we take the total population we also give information about how many people are not active in the country, which is not what we are interested in.

In [25]:
## Topology and basic map
    # Import the topo
europe_topo_path = 'topojson\europe.topojson.json'
europe_topo_data = json.load(open(europe_topo_path))

    # Clean the topo
europe_countries = europe_topo_data['objects']['europe']['geometries']
europe_countries.remove([country for country in europe_countries if country['properties']['NAME']=='Israel'][0])

# Create the Europe map
#europe_map = folium.Map([56.8,23], tiles='cartodbpositron', zoom_start=3)
#folium.TopoJson(europe_topo_data,'objects.europe').add_to(europe_map)

## Data
    # Import the data
europe_db_path = r'topojson\une_rt_a_1_Data.csv'
europe_db = pd.read_csv(europe_db_path)

    # Clean the data
europe_db['Value'] = europe_db['Value'].apply(lambda x: np.nan if x==':' else x)
europe_db['Value'] = pd.to_numeric(europe_db['Value'])

rename_dict = {'Germany (until 1990 former territory of the FRG)':'Germany'}
europe_db['GEO'] = europe_db['GEO'].apply(lambda x: rename_dict[x] if x in rename_dict.keys() else x)

In [26]:
## Function creating the coloured Choropleth map of unemployment
def unemployment_map(year) :
    """Displays the map of unemployment in Europe for the given year"""
    
        # Create map
    europe_map = folium.Map([56.8,23], tiles='cartodbpositron', zoom_start=3)
    
        # Select Data
    db = europe_db[europe_db['TIME']==year].set_index('GEO')['Value'].dropna()
    
        # Define a colormap depending on the quantiles
    quantiles = [db.min(),db.quantile(0.25),db.quantile(0.5),db.quantile(0.75),db.max()]
    colors = ['#75FA00','#FAF300','#FAA600','#D51410','#D51410']
    colormap = cm.StepColormap(colors,quantiles,vmin=db.min(),vmax=db.max())
    colormap.caption = 'Unemployment color scale'
    
        # Colour function
    def europe_colors(country):
        """Returns the colour depending on the unemployment rate, white if undefined"""
        try:
            return colormap(db[country])
        except:
            return '#FFFFFF'
    
        # Add the colours to the map
    folium.TopoJson(europe_topo_data, 'objects.europe',
            style_function= lambda feature : {
            'fillColor' : europe_colors(feature['properties']['NAME']),
            'fillOpacity' : 1,
            'color' : 'black',
            'weight' : 1} 
            ).add_to(europe_map)
    colormap.add_to(europe_map)
    return europe_map

In [27]:
unemployment_map(2011)

## Question 2: Unemployment rate in Switzerland

In [28]:
# Import the data and the topo
ch_topo_path = 'topojson\ch-cantons.topojson.json'
ch_topo_data = json.load(open(ch_topo_path))

# Create the Europe map
ch_map = folium.Map([46.85,8.15], tiles='cartodbpositron', zoom_start=7)
folium.TopoJson(ch_topo_data,'objects.cantons').add_to(ch_map)
ch_map